<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/resume_judger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/neuml/judge-your-resume-with-ai-4223a2803509

In [1]:
!pip install -q pypdf --progress-bar off
!pip install -qU langchain --progress-bar off
!pip install -qU transformers --progress-bar off
# Install AWQ version of model
!pip install -qU vllm --progress-bar off

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.


In [2]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# Store the model in our drive so we can load this model next time faster
# Make sure your drive has free space otherwise comment this.
os.environ['HF_HOME'] = '/content/drive/MyDrive/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/huggingface/models'

In [ ]:
from langchain.llms import VLLM

llm = VLLM(
    model="TheBloke/Starling-LM-7B-alpha-AWQ",
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=4096,
    top_k=10,
    top_p=0.95,
    temperature=0.8,
)

In [11]:
import langchain
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

In [6]:
from google.colab import files
resume = files.upload()

Saving Alpuerto_Joshua_C_CV.pdf to Alpuerto_Joshua_C_CV.pdf


In [7]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Alpuerto_Joshua_C_CV.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda

prompt = PromptTemplate.from_template(
    """GPT4 User:
You are a friendly assistant. You answer questions from users.

Answer the following question using only the context below.
Only include information specifically discussed.

question: {question}
context: {context} <|end_of_turn|>GPT4 Assistant:
"""
)

# function that will allow us to run this to a langchain expression
def llm_lambda(prompt):
    return llm.generate(prompts=[prompt.text], max_tokens=4000)


def vllm_output_parser(llm_results):
    # vllm generate list of generated result but on our usecase we only need one
    return llm_results.generations[0][0].text


llm_component = RunnableLambda(llm_lambda)
vllm_output_parser_component = RunnableLambda(vllm_output_parser)

runnable = prompt | llm_component | vllm_output_parser_component

In [55]:
questions = [
    "Who is Joshua?",
    "When did Joshua started working for Jobbatical?",
    "When did Joshua started working for Offshorly and when did it end?",

]

context = [page.page_content for page in pages]

for question in questions:
  runnable.invoke({"question": question, "context": context})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Who is Joshua?",
  "context": [
    "JoshuaC.Alpuerto\nE-MAILADDRESS:joshuacalpuerto@gmail.comMOBILE/TELEPHONE:37254005969URL:https://github.com/joshuaalpuerto/https://www.linkedin.com/in/joshua-alpuerto/https://www.polywork.com/joshuacalpuerto\nStaffEngineer\n FullSoftwareLifecycle\n▪JavascriptDevelopment\n▪EnterpriseDevelopment\n▪MobileDevelopment\n \nAs e n i o rs o f t w a r ee n g i n e e rw i t ho v e r1 0y e a r sc o m m e r c i a le x p e r i e n c ei ns o f t w a r ed e v e l o p m e n t .E x p e r i e n c ei nl e a d i n gt e a m s ,m e n t o r i n ge n g i n e e r sa n ds o f t w a r ea r c h i t e c t u r e .\nEMPLOYMENTRECORDS\n●JobbaticalTallinn,EstoniaJun2019–Present\n○Staffengineer\n■Involvedinprojectplanningandpreparationtoaidinclosingknowledgegapswithintheteam,allowingforquickerdeliveryandmoreproductivity. \n■Suggestedandimplementedtoolsandtechnologytoimproveourworkflows,therebyin

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.52s/it]


[llm/end] [1:llm:VLLM] [5.53s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\nJoshua started working for Jobbatical in June 2019.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:RunnableSequence > 3:chain:llm_lambda] [5.53s] Exiting Chain run with output:
{
  "lc": 1,
  "type": "not_implemented",
  "id": [
    "langchain_core",
    "outputs",
    "llm_result",
    "LLMResult"
  ],
  "repr": "LLMResult(generations=[[Generation(text='\\nJoshua started working for Jobbatical in June 2019.')]], llm_output=None, run=[RunInfo(run_id=UUID('0947bd05-8375-4b06-8553-1545fcbe9f67'))])"
}
[chain/start] [1:chain:RunnableSequence > 4:chain:vllm_output_parser] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 4:chain:vllm_output_parser] s] Exiting Chain run with output:
{
  "output": "\nJoshua started working for Jobbatical in June 2019."
}
[c

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.11s/it]

[llm/end] [1:llm:VLLM] [6.12s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\nJoshua started working for Offshorly in May 2016 and it ended in May 2019.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:RunnableSequence > 3:chain:llm_lambda] [6.12s] Exiting Chain run with output:
{
  "lc": 1,
  "type": "not_implemented",
  "id": [
    "langchain_core",
    "outputs",
    "llm_result",
    "LLMResult"
  ],
  "repr": "LLMResult(generations=[[Generation(text='\\nJoshua started working for Offshorly in May 2016 and it ended in May 2019.')]], llm_output=None, run=[RunInfo(run_id=UUID('47c7b2c4-c86f-41c8-939f-3922316baad1'))])"
}
[chain/start] [1:chain:RunnableSequence > 4:chain:vllm_output_parser] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 4:chain:vllm_output_parser] s] Exiting Chain run with output:
{
  "output": "\nJoshua star